In [1]:
import pandas as pd
import numpy as np
import math
import warnings
warnings.filterwarnings('ignore')

raw_data = pd.read_csv("CompleteDataSet_training_competition_1500.csv", sep=",")

data = raw_data[['BeltAccelerometer x-axis', 'BeltAccelerometer y-axis', 'BeltAccelerometer z-axis', 'WristAccelerometer x-axis', 'WristAccelerometer y-axis', 'WristAccelerometer z-axis', 'BrainSensor', 'Activity']]
av = raw_data[['BeltAngularVelocity x-axis', 'BeltAngularVelocity y-axis', 'BeltAngularVelocity z-axis', 'WristAngularVelocity x-axis', 'WristAngularVelocity y-axis', 'WristAngularVelocity z-axis']]

n_step = 1500
n_batch = int(data.shape[0] / n_step)

# variables used in the paper
k = 15
t = 3
K = 300

xyz = 3

data = data.as_matrix()
av = av.as_matrix()
calculate = np.zeros([n_batch * int(k / t), 95 + 1])

# for each data
for i in range(n_batch):
    
    # for each sliding window
    for j in range(int(k / t)):
        calculate[int(k / t) * i + j][95] = data[i * n_step][xyz+xyz+1]
        
        # belt variables
        belt_x = np.zeros([K])
        belt_y = np.zeros([K])
        belt_z = np.zeros([K])
        belt_a = np.zeros([K])
        belt_sa = 0
        belt_se = np.zeros([xyz])
        fft_belt_x = np.zeros([K])
        fft_belt_y = np.zeros([K])
        fft_belt_z = np.zeros([K])
        fft_belt_a = np.zeros([K])
        sqr_belt_x = np.zeros([K])
        sqr_belt_y = np.zeros([K])
        sqr_belt_z = np.zeros([K])
        sqr_belt_a = np.zeros([K])
        belt_av = np.zeros([xyz+1])
        psd_belt_x = 0
        psd_belt_y = 0
        psd_belt_z = 0
        psd_belt_a = 0
        belt_entropy = np.zeros([xyz+1])
        belt_corr = np.zeros((xyz, xyz))
        
        # wrist variables
        wrist_x = np.zeros([K])
        wrist_y = np.zeros([K])
        wrist_z = np.zeros([K])
        wrist_a = np.zeros([K])
        wrist_sa = 0
        wrist_se = np.zeros([xyz])
        fft_wrist_x = np.zeros([K])
        fft_wrist_y = np.zeros([K])
        fft_wrist_z = np.zeros([K])
        fft_wrist_a = np.zeros([K])
        sqr_wrist_x = np.zeros([K])
        sqr_wrist_y = np.zeros([K])
        sqr_wrist_z = np.zeros([K])
        sqr_wrist_a = np.zeros([K])
        wrist_av = np.zeros([xyz+1])
        psd_wrist_x = 0
        psd_wrist_y = 0
        psd_wrist_z = 0
        psd_wrist_a = 0
        wrist_entropy = np.zeros([xyz+1])
        wrist_corr = np.zeros((xyz, xyz))
        
        # brainwave variables
        brainwave = np.zeros([K])
        brainwave_sa = 0
        fft_brainwave = np.zeros([K])
        sqr_brainwave = np.zeros([K])
        psd_brainwave = 0
        brainwave_entropy = 0     
        
        for m in range(K):                
            belt_x[m] = data[i * n_step + j * K + m][0]
            belt_y[m] = data[i * n_step + j * K + m][1]
            belt_z[m] = data[i * n_step + j * K + m][2]
            belt_a[m] = np.sqrt(pow(data[i * n_step + j * K + m][0], 2) + pow(data[i * n_step + j * K + m][1], 2) + pow(data[i * n_step + j * K + m][2], 2))
            belt_sa += (np.abs(belt_x[m]) + np.abs(belt_y[m]) + np.abs(belt_z[m]))
            belt_av[0] += av[i * n_step + j * K + m][0]
            belt_av[1] += av[i * n_step + j * K + m][1]
            belt_av[2] += av[i * n_step + j * K + m][2]
            belt_av[3] += np.sqrt(pow(av[i * n_step + j * K + m][0], 2) + pow(av[i * n_step + j * K + m][1], 2) + pow(av[i * n_step + j * K + m][2], 2))
            wrist_x[m] = data[i * n_step + j * K + m][3]
            wrist_y[m] = data[i * n_step + j * K + m][4]
            wrist_z[m] = data[i * n_step + j * K + m][5]
            wrist_a[m] = np.sqrt(pow(data[i * n_step + j * K + m][3], 2) + pow(data[i * n_step + j * K + m][4], 2) + pow(data[i * n_step + j * K + m][5], 2))
            wrist_sa += (np.abs(wrist_x[m]) + np.abs(wrist_y[m]) + np.abs(wrist_z[m]))
            wrist_av[0] += av[i * n_step + j * K + m][3]
            wrist_av[1] += av[i * n_step + j * K + m][4]
            wrist_av[2] += av[i * n_step + j * K + m][5]
            wrist_av[3] += np.sqrt(pow(av[i * n_step + j * K + m][3], 2) + pow(av[i * n_step + j * K + m][4], 2) + pow(av[i * n_step + j * K + m][5], 2))
            brainwave[m] = data[i * n_step + j * K + m][6]
            brainwave_sa += np.abs(brainwave[m])
        
        fft_belt_x = np.fft.fft(belt_x)
        fft_belt_y = np.fft.fft(belt_y)
        fft_belt_z = np.fft.fft(belt_z)
        fft_belt_a = np.fft.fft(belt_a)
        fft_wrist_x = np.fft.fft(wrist_x)
        fft_wrist_y = np.fft.fft(wrist_y)
        fft_wrist_z = np.fft.fft(wrist_z)
        fft_wrist_a = np.fft.fft(wrist_a)
        fft_brainwave = np.fft.fft(brainwave)
        for m in range(K):
            belt_se[0] += pow(fft_belt_x[m], 2)
            belt_se[1] += pow(fft_belt_y[m], 2)
            belt_se[2] += pow(fft_belt_z[m], 2)
            wrist_se[0] += pow(fft_wrist_x[m], 2)
            wrist_se[1] += pow(fft_wrist_y[m], 2)
            wrist_se[2] += pow(fft_wrist_z[m], 2)
            
            sqr_belt_x[m] = np.sqrt(fft_belt_x[m])
            sqr_belt_y[m] = np.sqrt(fft_belt_y[m])
            sqr_belt_z[m] = np.sqrt(fft_belt_z[m])
            sqr_belt_a[m] = np.sqrt(fft_belt_a[m])
            sqr_wrist_x[m] = np.sqrt(fft_wrist_x[m])
            sqr_wrist_y[m] = np.sqrt(fft_wrist_y[m])
            sqr_wrist_z[m] = np.sqrt(fft_wrist_z[m])
            sqr_wrist_a[m] = np.sqrt(fft_wrist_a[m])
            sqr_brainwave = np.sqrt(fft_brainwave)
        
        psd_belt_x = np.mean(sqr_belt_x)
        psd_belt_y = np.mean(sqr_belt_y)
        psd_belt_z = np.mean(sqr_belt_z)
        psd_belt_a = np.mean(sqr_belt_a)
        psd_wrist_x = np.mean(sqr_wrist_x)
        psd_wrist_y = np.mean(sqr_wrist_y)
        psd_wrist_z = np.mean(sqr_wrist_z)
        psd_wrist_a = np.mean(sqr_wrist_a)
        psd_brainwave = np.mean(sqr_brainwave)
        belt_entropy[0] = psd_belt_x * np.log2(psd_belt_x)
        belt_entropy[1] = psd_belt_y * np.log2(psd_belt_y)
        belt_entropy[2] = psd_belt_z * np.log2(psd_belt_z)
        belt_entropy[3] = psd_belt_a * np.log2(psd_belt_a)
        wrist_entropy[0] = psd_wrist_x * np.log2(psd_wrist_x)
        wrist_entropy[1] = psd_wrist_y * np.log2(psd_wrist_y)
        wrist_entropy[2] = psd_wrist_z * np.log2(psd_wrist_z)
        wrist_entropy[3] = psd_wrist_a * np.log2(psd_wrist_a)
        brainwave_entropy = psd_brainwave * np.log2(psd_brainwave)
        
        belt_lst = [belt_x, belt_y, belt_z]
        belt_df = pd.DataFrame(belt_lst).T
        belt_corr = belt_df.corr(method = 'pearson')
        wrist_lst = [wrist_x, wrist_y, wrist_z]
        wrist_df = pd.DataFrame(wrist_lst).T
        wrist_corr = wrist_df.corr(method = 'pearson')
        
        calculate[int(k / t) * i + j][0] = np.std(belt_x)
        calculate[int(k / t) * i + j][1] = np.std(belt_y)
        calculate[int(k / t) * i + j][2] = np.std(belt_z)
        calculate[int(k / t) * i + j][3] = np.std(belt_a)
        calculate[int(k / t) * i + j][4] = np.mean(belt_x)
        calculate[int(k / t) * i + j][5] = np.mean(belt_y)
        calculate[int(k / t) * i + j][6] = np.mean(belt_z)
        calculate[int(k / t) * i + j][7] = np.mean(belt_a)
        calculate[int(k / t) * i + j][8] = max(belt_x)
        calculate[int(k / t) * i + j][9] = max(belt_y)
        calculate[int(k / t) * i + j][10] = max(belt_z)
        calculate[int(k / t) * i + j][11] = max(belt_a)
        calculate[int(k / t) * i + j][12] = min(belt_x)
        calculate[int(k / t) * i + j][13] = min(belt_y)
        calculate[int(k / t) * i + j][14] = min(belt_z)
        calculate[int(k / t) * i + j][15] = min(belt_a)
        calculate[int(k / t) * i + j][16] = belt_sa
        calculate[int(k / t) * i + j][17] = np.median(belt_x)
        calculate[int(k / t) * i + j][18] = np.median(belt_y)
        calculate[int(k / t) * i + j][19] = np.median(belt_z)
        calculate[int(k / t) * i + j][20] = np.median(belt_a)
        calculate[int(k / t) * i + j][21] = np.percentile(belt_x, 25)
        calculate[int(k / t) * i + j][22] = np.percentile(belt_y, 25)
        calculate[int(k / t) * i + j][23] = np.percentile(belt_z, 25)
        calculate[int(k / t) * i + j][24] = np.percentile(belt_a, 25)
        calculate[int(k / t) * i + j][25] = np.percentile(belt_x, 75)
        calculate[int(k / t) * i + j][26] = np.percentile(belt_y, 75)
        calculate[int(k / t) * i + j][27] = np.percentile(belt_z, 75)
        calculate[int(k / t) * i + j][28] = np.percentile(belt_a, 75)
        calculate[int(k / t) * i + j][29] = belt_se[0]
        calculate[int(k / t) * i + j][30] = belt_se[1]
        calculate[int(k / t) * i + j][31] = belt_se[2]
        calculate[int(k / t) * i + j][32] = belt_av[0] / K
        calculate[int(k / t) * i + j][33] = belt_av[1] / K
        calculate[int(k / t) * i + j][34] = belt_av[2] / K
        calculate[int(k / t) * i + j][35] = belt_av[3] / K
        calculate[int(k / t) * i + j][36] = belt_entropy[0]
        calculate[int(k / t) * i + j][37] = belt_entropy[1]
        calculate[int(k / t) * i + j][38] = belt_entropy[2]
        calculate[int(k / t) * i + j][39] = belt_entropy[3]
        calculate[int(k / t) * i + j][40] = belt_corr[0][1]
        calculate[int(k / t) * i + j][41] = belt_corr[1][2]
        calculate[int(k / t) * i + j][42] = belt_corr[0][2]        
        calculate[int(k / t) * i + j][43] = np.std(wrist_x)
        calculate[int(k / t) * i + j][44] = np.std(wrist_y)
        calculate[int(k / t) * i + j][45] = np.std(wrist_z)
        calculate[int(k / t) * i + j][46] = np.std(wrist_a)
        calculate[int(k / t) * i + j][47] = np.mean(wrist_x)
        calculate[int(k / t) * i + j][48] = np.mean(wrist_y)
        calculate[int(k / t) * i + j][49] = np.mean(wrist_z)
        calculate[int(k / t) * i + j][50] = np.mean(wrist_a)
        calculate[int(k / t) * i + j][51] = max(wrist_x)
        calculate[int(k / t) * i + j][52] = max(wrist_y)
        calculate[int(k / t) * i + j][53] = max(wrist_z)
        calculate[int(k / t) * i + j][54] = max(wrist_a)
        calculate[int(k / t) * i + j][55] = min(wrist_x)
        calculate[int(k / t) * i + j][56] = min(wrist_y)
        calculate[int(k / t) * i + j][57] = min(wrist_z)
        calculate[int(k / t) * i + j][58] = min(wrist_a)
        calculate[int(k / t) * i + j][59] = wrist_sa
        calculate[int(k / t) * i + j][60] = np.median(wrist_x)
        calculate[int(k / t) * i + j][61] = np.median(wrist_y)
        calculate[int(k / t) * i + j][62] = np.median(wrist_z)
        calculate[int(k / t) * i + j][63] = np.median(wrist_a)
        calculate[int(k / t) * i + j][64] = np.percentile(wrist_x, 25)
        calculate[int(k / t) * i + j][65] = np.percentile(wrist_y, 25)
        calculate[int(k / t) * i + j][66] = np.percentile(wrist_z, 25)
        calculate[int(k / t) * i + j][67] = np.percentile(wrist_a, 25)
        calculate[int(k / t) * i + j][68] = np.percentile(wrist_x, 75)
        calculate[int(k / t) * i + j][69] = np.percentile(wrist_y, 75)
        calculate[int(k / t) * i + j][70] = np.percentile(wrist_z, 75)
        calculate[int(k / t) * i + j][71] = np.percentile(wrist_a, 75)
        calculate[int(k / t) * i + j][72] = wrist_se[0]
        calculate[int(k / t) * i + j][73] = wrist_se[1]
        calculate[int(k / t) * i + j][74] = wrist_se[2]
        calculate[int(k / t) * i + j][75] = wrist_av[0] / K
        calculate[int(k / t) * i + j][76] = wrist_av[1] / K
        calculate[int(k / t) * i + j][77] = wrist_av[2] / K
        calculate[int(k / t) * i + j][78] = wrist_av[3] / K
        calculate[int(k / t) * i + j][79] = wrist_entropy[0]
        calculate[int(k / t) * i + j][80] = wrist_entropy[1]
        calculate[int(k / t) * i + j][81] = wrist_entropy[2]
        calculate[int(k / t) * i + j][82] = wrist_entropy[3]
        calculate[int(k / t) * i + j][83] = wrist_corr[0][1]
        calculate[int(k / t) * i + j][84] = wrist_corr[1][2]
        calculate[int(k / t) * i + j][85] = wrist_corr[0][2]
        calculate[int(k / t) * i + j][86] = np.std(brainwave)
        calculate[int(k / t) * i + j][87] = np.mean(brainwave)
        calculate[int(k / t) * i + j][88] = max(brainwave)
        calculate[int(k / t) * i + j][89] = min(brainwave)
        calculate[int(k / t) * i + j][90] = brainwave_sa
        calculate[int(k / t) * i + j][91] = np.median(brainwave)
        calculate[int(k / t) * i + j][92] = np.percentile(brainwave, 25)
        calculate[int(k / t) * i + j][93] = np.percentile(brainwave, 75)
        calculate[int(k / t) * i + j][94] = brainwave_entropy
        
data = pd.DataFrame(calculate)

data.to_csv("./dataset/training_data_LSTM_FS_3_BeltWristBW.csv", sep=",", header=False, index=False)